In [193]:
#Liste des imports

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim


In [194]:
#Récupérer les données 

label = np.load("Label.npy")
targetBac = np.load("targetBac.npy")
targetK7 =  np.load("targetK7.npy")


print(len(label))
print(type(label[0][0]))

13423
<class 'numpy.int32'>


In [195]:
#Paramètres

input_size= len(label[0])
hidden_dim = 1000
n_layers = 10
output_size = len(label[0])

batch_size = 200
num_epochs = 1000
n_iter = num_epochs * (len(targetBac) / batch_size)
n_iter = int(n_iter)


In [196]:
#Prep input bac
input_seq = []
targetBac_seq = []
for i in range(len(label)-1):
    #input_seq.append([label[i],targetBac[i]])
    #targetBac_seq.append([label[i+1],targetBac[i+1]])
    input_seq.append(np.concatenate((label[i],targetBac[i]),axis=0))
    targetBac_seq.append(np.concatenate((label[i+1],targetBac[i+1]),axis=0))
    
input_seq = torch.from_numpy(np.asarray(input_seq))
targetBac_seq = torch.from_numpy(np.asarray(targetBac_seq))
print(input_seq.size())

#Adding an axis to input 
input_seq = input_seq[None,:,:]
targetBac_seq = targetBac_seq[None,:,:]

input_seq  = input_seq.float()
targetBac_seq = targetBac_seq.float()


print(input_seq.size())

torch.Size([13422, 264])
torch.Size([1, 13422, 264])


In [197]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [198]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        # Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers)
        # FC Layer
        self.fc = nn.Linear(hidden_dim, output_size)


    def forward(self, x):
        batch_size = x.size(1)

        # Initializing hidden state for thr first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining output
        out, hidden = self.rnn(x, hidden)

        # Reshaping the outputs such that it can be fit into the FC Layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)


        return out, hidden

    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch .zeros(self.n_layers, batch_size, self.hidden_dim)
        return hidden

In [199]:
model = Model(input_size=input_seq.size(2) , output_size=targetBac_seq.size(2), hidden_dim=hidden_dim, n_layers=n_layers)
# We'll also set the model to the device that we defined earlier (default is CPU)
model.to(device)


# Define Loss, Optimizer
lr=0.01
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
# Training Run 
Loss = []
Epoch = []
for epoch in range(1, num_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    input_seq.to(device)
    output, hidden = model(input_seq)
    loss = criterion(output, targetBac_seq[0])
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step()

    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, num_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))
        Loss.append(loss.item)
        Epoch.append(epoch)
        
        

Epoch: 10/1000............. Loss: 6798.0068
Epoch: 20/1000............. Loss: 6016.6626
Epoch: 30/1000............. Loss: 5324.3774
Epoch: 40/1000............. Loss: 4710.2407
Epoch: 50/1000............. Loss: 4165.6167
Epoch: 60/1000............. Loss: 3683.4792
Epoch: 70/1000............. Loss: 3257.8630
Epoch: 80/1000............. Loss: 2883.4846
Epoch: 90/1000............. Loss: 2555.5105
Epoch: 100/1000............. Loss: 2269.4370
Epoch: 110/1000............. Loss: 2021.0436
Epoch: 120/1000............. Loss: 1806.3679
Epoch: 130/1000............. Loss: 1621.7145
Epoch: 140/1000............. Loss: 1463.6537
Epoch: 150/1000............. Loss: 1329.0129
Epoch: 160/1000............. Loss: 1214.8931
Epoch: 170/1000............. Loss: 1118.6527
Epoch: 180/1000............. Loss: 1037.8900
Epoch: 190/1000............. Loss: 970.4661
Epoch: 200/1000............. Loss: 914.4666
Epoch: 210/1000............. Loss: 868.1942
Epoch: 220/1000............. Loss: 830.1541
Epoch: 230/1000........

In [ ]:
#Plot results
import matplotlib.pyplot as plt
plt.plot(Epoch,Loss)
plt.show

In [ ]:
print(input_seq.size())

In [ ]:
print(targetBac_seq.long())